In [1]:
import os
import sys
import pandas as pd
import subprocess
import shutil
import numpy as np
from pathlib import Path

In [2]:
root_dir = "/nfs/turbo/umms-indikar/shared/projects/poreC/data/raw_sequencing_files/"

barcodes = [f"barcode{str(x).zfill(2)}" for x in range(1, 97)]

run_files = {}


# collect all the files to merge
for subdir in os.listdir(root_dir):
    subdir_path = f"{root_dir}{subdir}/"

    if not subdir in run_files:
        run_files[subdir] = {}
    
    for pod5_dir in os.listdir(subdir_path):
        source_path = f"{root_dir}{subdir}/{pod5_dir}/"

        for barcode in barcodes:
            barcode_path = f"{source_path}{barcode}/"
            
            if  not os.path.isdir(barcode_path):
                continue
            
            file_list = os.listdir(barcode_path)
            file_path_list = [f"{barcode_path}{x}" for x in file_list]

            if not barcode in run_files[subdir]:
                run_files[subdir][barcode] = file_path_list
            else:
                run_files[subdir][barcode].extend(file_path_list)

print('done')

done


In [3]:
# GLOBAL VARS
outpath = "/nfs/turbo/umms-indikar/shared/projects/poreC/data/raw_sequencing_files/"
new_dir_name = "merged_pod5/"
pod5 = "~/miniconda3/envs/scpc/bin/pod5"
threads = 36

# LAUNCHER PARAMS

# build the launcher script
sub_commands = os.path.abspath("launcher_scripts/")
job_file_path = 'merge_pod5_launch.sh'

try:
    os.remove(job_file_path)
except OSError:
    pass

# build the launcher script and sub_commands iteratively
for run, barcodes in run_files.items():
    for barcode, file_list in barcodes.items():
        outname = f"{outpath}{run}/{new_dir_name}{barcode}.pod5"

        cmd = f"{pod5} merge {' '.join(file_list)} --threads {threads} -f --output {outname}"

        # add file to job list
        launcher_command_path = f"{sub_commands}/{run}_{barcode}.sh"
        print(launcher_command_path, file=open(job_file_path, "a"))

        try:
            os.remove(launcher_command_path)
        except OSError:
            pass

        # build individual sub_commands
        print(cmd, file=open(launcher_command_path, "a"))

print('done')

done
